In [131]:
from google import genai
from google.genai import types
import os
import json
from dotenv import load_dotenv

In [132]:
load_dotenv()

API_KEY = os.getenv("GEMINI_API_KEY")

if not API_KEY:
    raise ValueError("GEMINI_API_KEY not found.")

In [133]:
chat_history= []

def history_append(role: str, text: str):
  global chat_history
  chat_history.append(types.Content(
    role=role,
    parts=[types.Part.from_text(text=text)]
  ))

In [134]:
MODEL = "gemini-2.5-flash"
SYSTEM_INSTRUCTION = {
    "You are an AI assistant for a financial services company. "
    "Your role is to answer user queries about finance, banking, and investments. "
    "Be polite, clear, and professional."
}
CONFIG = types.GenerateContentConfig(
  system_instruction=SYSTEM_INSTRUCTION,
  temperature=0.3,
)

history_append("system", "You are an AI assistant for a financial services company. "
                         "Your role is to answer user queries about finance, banking, and investments. "
                         "Be polite, clear, and professional.")



client = genai.Client(api_key=API_KEY)

In [135]:
# Create new chat
chat = client.chats.create(model = MODEL, config= CONFIG)

In [136]:
def give_prompt(text_prompt:str)->str:
  global chat_history
  global chat
  
  history_append("user", text_prompt)

  response = chat.send_message(text_prompt)

  history_append("model", response.text)

  return response.text


In [137]:
print("What is my name")
print(give_prompt("what is my name?"))


print(give_prompt("what was my lasst question?"))

What is my name
I apologize, but as an AI assistant, I do not have access to your personal information, including your name. My purpose is to provide information and answer questions about finance, banking, and investments.
Your last question was: "what is my name?"


In [138]:
print(chat_history)

[Content(
  parts=[
    Part(
      text='You are an AI assistant for a financial services company. Your role is to answer user queries about finance, banking, and investments. Be polite, clear, and professional.'
    ),
  ],
  role='system'
), Content(
  parts=[
    Part(
      text='what is my name?'
    ),
  ],
  role='user'
), Content(
  parts=[
    Part(
      text='I apologize, but as an AI assistant, I do not have access to your personal information, including your name. My purpose is to provide information and answer questions about finance, banking, and investments.'
    ),
  ],
  role='model'
), Content(
  parts=[
    Part(
      text='what was my lasst question?'
    ),
  ],
  role='user'
), Content(
  parts=[
    Part(
      text='Your last question was: "what is my name?"'
    ),
  ],
  role='model'
)]


In [139]:
client.close()

In [140]:
EXTRACTOR_API_KEY = os.getenv("GEMINI_EXTRACTOR_API_KEY")

if not EXTRACTOR_API_KEY:
    raise ValueError("GEMINI_API_KEY not found.")

extractor = genai.Client(api_key=EXTRACTOR_API_KEY)
# extractor_chat = extractor.chats.create(model=MODEL)

In [141]:
user_db = []

In [142]:
def extract_details(text_prompt:str)->dict:
  global extractor

  extraction_prompt = f"""
      Analyze the following text from a user. Extract any important personal
      details such as name, account number, specific financial goal, 
      or contact information. 
      
      Respond in JSON format with keys "name", "account_number", "goal", 
      "contact", or "other_details". If no information is found for a key, 
      use null.

      Text: "{text_prompt}"
      
      JSON:
      """
  
  try:
    response = client.models.generate_content(
      model=MODEL, 
      contents = [extraction_prompt], 
      config=types.GenerateContentConfig(
        response_mime_type="application/json",
      ),
    )

    return json.loads(response.text)
  
  except Exception as e:
    print(f"[Detail Extraction Failed: {e}]")
    return None


In [143]:
new_details = extract_details("my name is Arpit Raj. Im a student. My account number is 1234")

[Detail Extraction Failed: Cannot send a request, as the client has been closed.]


In [144]:
print(new_details)
print(type(new_details))

None
<class 'NoneType'>


In [145]:
def update_db(new_details):
  global user_db
  has_new_info = False
  
  for new_detail in new_details:
        for key, value in new_details.items():
            if value and user_db.get(key) != value:
                user_db[key] = value
                has_new_info = True

  return has_new_info 

In [146]:
user_db = {"Roll": "2301EC07", "name" :"ayush", "add": "Patna"}

new_details={'name': 'Arpit Raj', 'account_number': '1234', 'goal': None, 'contact': None, 'other_details': 'student'}

In [147]:
has_new = update_db(new_details)

In [148]:
user_db

{'Roll': '2301EC07',
 'name': 'Arpit Raj',
 'add': 'Patna',
 'account_number': '1234',
 'other_details': 'student'}